# Models

> Models Available For This Project

In [ ]:
#| default_exp scripts.modeling

In [ ]:
#| hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#| export

from fastcore.script import Param, call_parse

from sklearn import preprocessing as processors
from sklearn.model_selection import train_test_split

from LTBP.data.utils import snowflake_query, get_yaml_dicts, generate_data_lake_query
from LTBP.modeling.utils import (
    create_stage_and_query_stage_sf, create_sklearn_preprocess_baseline_dict, 
    return_list_of_vars, prepare_training_set
)
from LTBP.modeling.custom_utils import evaluate
from LTBP import files

from machine_learning_utilities import preprocessing

from data_system_utilities.file_parsers import yaml
from data_system_utilities.snowflake.utils import make_stage_query_generator

from sklearn import metrics
from sklearn import preprocessing as processors
from sklearn.model_selection import train_test_split

import LTBP.modeling.models as ds_models
import pandas as pd
import sys
import pandas as pd
import sys
import os
import logging

/Users/jeremydemlow/miniforge3/envs/ltbp/lib/python3.9/site-packages/snowflake/connector/options.py:96: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.0), please install a version that adheres to: 'pyarrow<8.1.0,>=8.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [ ]:
#| hide
from pathlib import Path
import yaml

In [ ]:
#| hide
def write_yaml_file(file_path: str, file_name: str, dictionary: dict):
    with open(Path(file_path, file_name), 'w') as f:
        yaml.dump(dictionary, f)

models = dict({
    'preprocessors_adls_path' : 'preprocessors/',
    'connection_str': 'DATALAKE_CONN_STR_SECRET',
    'hold_out_table' : 'LTBP_HOLDOUT_TEST_MODEL_RESULTS',
    'tracking_table' : 'LTBP_MODEL_TRACKING_FY23',
    'idenfication': ['ECID', 'SEASONYEAR'],
    'BASELINE': {
        'description': 'Standard baseline xgb_hyperopt approach status quo of LTBP of the past',
        'model_trainer': 'train_xgb',
        'y_preprocess_object_name': None,
        'y_scaler_type' : None,
        'x_preprocess_object_name': 'standard_pipe.pickle'

    },
    'NOHYPEROPT': {
        'description': 'Only here to see if it works delete at some point xgb_fit_only',
        'model_trainer': 'train_xgb_basic',
        'y_preprocess_object_name': None,
        'y_scaler_type' : None,
        'x_preprocess_object_name': 'standard_pipe.pickle'
    }
})

write_yaml_file('./LTBP/files/yaml_files/', 'models.yaml', models)

This is the DSDE standard process for using Xboost with hyperopt

In [ ]:
#| export


@call_parse
def model_train(yaml_file_list: Param(help="YAML files to read", type=list, # noqa:
                                      default=['dataset.yaml', 'etl.yaml', 'experiment.yaml']),  # noqa:
                hyper_sub_size: Param(help="", type=int, default=2750000),  # noqa:
                train_size: Param(help="", type=int, default=5500000),  # noqa:
                evals: Param(help="logit threshold cut", type=int, default=10),  # noqa:
                train_model: Param(help="to subsample or not", type=str, default='True'),  # noqa:
                fit_only: Param(help="no tuning model training", type=str, default='False')):  # noqa:
    
    features, udf_inputs, etl, models = get_yaml_dicts(['features.yaml', 'udf_inputs.yaml', 'etl.yaml', 'models.yaml'])
    df = create_stage_and_query_stage_sf(
        sf=sf,
        etl=etl,
        udf_inputs=udf_inputs,
        train_or_inference=train_or_inference,
        experiment_name=experiment_name,
        experiment=experiment,
        indentification=models['idenfication']
        )

    cat_vars =[{f.upper() : values['transformation'][experiment_name]} for f, values in features.items() 
                if values['var_type'][experiment_name] == 'cat'
                and values['input_definition'] != 'LABEL']
    cont_vars =[{f.upper(): values['transformation'][experiment_name]} for f, values in features.items() 
                if values['var_type'][experiment_name] == 'cont'
                and values['input_definition'] != 'LABEL']

    feature_dict = create_sklearn_preprocess_baseline_dict(cat_vars=cat_vars, 
                                                           cont_vars=cont_vars)
    logging.info(feature_dict)
    cat_vars = return_list_of_vars(cat_vars)
    cont_vars = return_list_of_vars(cont_vars)
    logging.info(f"categorical variables: \n {cat_vars}")
    logging.info(f"continous variables: \n {cont_vars}")

    pipe = preprocessing.generate_sklearn_preprocessing_pipeline(
        feature_dict, impute=True, impute_strategy='mean'
    )

    adls_path = os.path.join((os.path.join(etl['data_lake_path'], 'experiments', experiment_name)
        if experiment 
        else os.path.join(etl_dict['data_lake_path'], 
        os.environ.get('CI_COMMIT_SHA', 'LocalRunNBS')))
        , models['preprocessors_adls_path'])

    y_var=[k.upper() for k, v in features.items() if v['input_definition'] == 'LABEL']

    result = prepare_training_set(df,
                                  y_var=[k.upper() for k, v in features.items() if v['input_definition'] == 'LABEL'],
                                  y_scaler_type=models[experiment_name]['y_scaler_type'],
                                  adls_path=adls_path,
                                  sklearn_pipe=pipe,
                                  test_set=True,
                                  etl_dict=etl,
                                  models_dict=models,
                                  connection_str=os.environ[models["connection_str"]],
                                  experiment_name=experiment_name,
                                  as_type=int,
                                  identifiers=['ECID', 'SEASONYEAR']
                                  )
    if test_set:
        X_train, X_valid, X_test, y_train, y_valid, y_test, sklearn_pipe, scaler, id_list = result
    else:
        X_train, X_valid, y_train, y_valid, sklearn_pipe, scaler, id_list = result

    model_trainer = getattr(ds_models, models[experiment_name]['model_trainer'])

    cat_vars =[{f.upper() : values['transformation'][experiment_name]} for f, values in features.items() 
                if values['var_type'][experiment_name] == 'cat'
                and values['input_definition'] != 'LABEL']
    cont_vars =[{f.upper(): values['transformation'][experiment_name]} for f, values in features.items() 
                if values['var_type'][experiment_name] == 'cont'
                and values['input_definition'] != 'LABEL']

    feature_dict = create_sklearn_preprocess_baseline_dict(cat_vars=cat_vars, 
                                                           cont_vars=cont_vars)
    logging.info(feature_dict)
    cat_vars = return_list_of_vars(cat_vars)
    cont_vars = return_list_of_vars(cont_vars)
    logging.info(f"categorical variables: \n {cat_vars}")
    logging.info(f"continous variables: \n {cont_vars}")

    pipe = preprocessing.generate_sklearn_preprocessing_pipeline(
        feature_dict, impute=True, impute_strategy='mean'
    )

    adls_path = os.path.join((os.path.join(etl['data_lake_path'], 'experiments', experiment_name)
        if experiment
        else os.path.join(etl_dict['data_lake_path'],
        os.environ.get('CI_COMMIT_SHA', 'LocalRunNBS')))
        , models['preprocessors_adls_path'])

    model = model_trainer(X_train, X_valid, y_train, y_valid,
        evals=evals, sub=hyper_sub_size, train=train_size)

    y_var=[k.upper() for k, v in features.items() if v['input_definition'] == 'LABEL']

    """
    Custom needs for each project type this works for a binary classification
    this is not my best work, but trying to put something together 
    this is dry I am sure i could make this just a few lines 
    """
    result_dict = {}
    logging.info('Training Set Evaluation')

    eval_list_train = evaluate(model, X_train, y_train, y_var, feature_importance=True)
    metric1, metric2, metric3, columns, fi_permutation = eval_list_train
    result_dict['training_metrics']={k: v for k, v in zip(columns, [metric1]+[metric2]+[metric3])}
    result_dict['fi_train']={k:v for k, v in fi_permutation[:10].values}
    logging.info('Validation Set Evaluation')
    eval_list_valid = evaluate(model, X_valid, y_valid, y_var, feature_importance=True)
    metric1, metric2, metric3, columns, fi_permutation = eval_list_train
    result_dict['valid_metrics']={k: v for k, v in zip(columns, [metric1]+[metric2]+[metric3])}
    result_dict['fi_valid']={k:v for k, v in fi_permutation[:10].values}
    if X_test is not None:
        logging.info('Test Set Evaluation')
        eval_list_test = evaluate(model, X_test, y_test, y_var, feature_importance=True)
        metric1, metric2, metric3, columns, fi_permutation = eval_list_train
        result_dict['test_metrics']={k: v for k, v in zip(columns, [metric1]+[metric2]+[metric3])}
        result_dict['fi_test']={k:v for k, v in fi_permutation[:10].values}

    sf = snowflake_query(sfSchema='LTBP')
    send_holdout_results_to_sf(sf, id_list, y_pred_proba, experiment, experiment_name, etl, models)

    adls_path = os.path.join((os.path.join(etl['data_lake_path'], 'experiments', experiment_name)
        if experiment 
        else os.path.join(etl_dict['data_lake_path'], 
        os.environ.get('CI_COMMIT_SHA', 'LocalRunNBS'))))

    project_log_df = project_log.project_log(
        snowflake_connection=sf,
        table_name=models['tracking_table'],
        action_description=models[experiment_name]["description"],
        transaction_type="model_training",
        artifacts=json.dumps({"azure_parent_folder": adls_path}),
        metrics=json.dumps(result_dict),
        append_or_replace="append",
    )
    logging.info(f'project log preview:\n{project_log_df.head(2)}')

In [ ]:
experiment = True # this will trigger if the feature set needs to be created
train_or_inference = 'TRAINING' # 'INFERENCE'
experiment_name='BASELINE'
sf = snowflake_query()

scaler_type = getattr(processors, 'LabelEncoder')() # Don't actuall use this with a binaryclass
scaler_type = None
y_var='BOUGHTPASS'
test_set=True
connection_str = os.environ['DATALAKE_CONN_STR_SECRET']
experiment_name='BASELINE'

evals=2
hyper_sub_size=200000
train_size=10000

In [ ]:
#| hide

features, udf_inputs, etl, models = get_yaml_dicts(['features.yaml',
                                                    'udf_inputs.yaml',
                                                    'etl.yaml', 
                                                    'models.yaml'])
df = create_stage_and_query_stage_sf(
    sf=sf,
    etl=etl,
    udf_inputs=udf_inputs,
    train_or_inference=train_or_inference,
    experiment_name=experiment_name,
    experiment=experiment,
    indentification=models['idenfication']
    )

cat_vars =[{f.upper() : values['transformation'][experiment_name]} for f, values in features.items() 
            if values['var_type'][experiment_name] == 'cat'
            and values['input_definition'] != 'LABEL']
cont_vars =[{f.upper(): values['transformation'][experiment_name]} for f, values in features.items() 
            if values['var_type'][experiment_name] == 'cont'
            and values['input_definition'] != 'LABEL']

feature_dict = create_sklearn_preprocess_baseline_dict(cat_vars=cat_vars, 
                                                       cont_vars=cont_vars)
logging.info(feature_dict)
cat_vars = return_list_of_vars(cat_vars)
cont_vars = return_list_of_vars(cont_vars)
logging.info(f"categorical variables: \n {cat_vars}")
logging.info(f"continous variables: \n {cont_vars}")

pipe = preprocessing.generate_sklearn_preprocessing_pipeline(
    feature_dict, impute=True, impute_strategy='mean'
)

adls_path = os.path.join((os.path.join(etl['data_lake_path'], 'experiments', experiment_name)
    if experiment 
    else os.path.join(etl_dict['data_lake_path'], 
    os.environ.get('CI_COMMIT_SHA', 'LocalRunNBS')))
    , models['preprocessors_adls_path'])

y_var=[k.upper() for k, v in features.items() if v['input_definition'] == 'LABEL']

result = prepare_training_set(df,
                              y_var=[k.upper() for k, v in features.items() if v['input_definition'] == 'LABEL'],
                              y_scaler_type=models[experiment_name]['y_scaler_type'],
                              adls_path=adls_path,
                              sklearn_pipe=pipe,
                              test_set=True,
                              etl_dict=etl,
                              models_dict=models,
                              connection_str=os.environ[models["connection_str"]],
                              experiment_name=experiment_name,
                              as_type=int,
                              identifiers=['ECID', 'SEASONYEAR']
                              )
if test_set:
    X_train, X_valid, X_test, y_train, y_valid, y_test, sklearn_pipe, scaler, id_list = result
else:
    X_train, X_valid, y_train, y_valid, sklearn_pipe, scaler, id_list = result

model_trainer = getattr(ds_models, models[experiment_name]['model_trainer'])

cat_vars =[{f.upper() : values['transformation'][experiment_name]} for f, values in features.items() 
            if values['var_type'][experiment_name] == 'cat'
            and values['input_definition'] != 'LABEL']
cont_vars =[{f.upper(): values['transformation'][experiment_name]} for f, values in features.items() 
            if values['var_type'][experiment_name] == 'cont'
            and values['input_definition'] != 'LABEL']

feature_dict = create_sklearn_preprocess_baseline_dict(cat_vars=cat_vars, 
                                                       cont_vars=cont_vars)
logging.info(feature_dict)
cat_vars = return_list_of_vars(cat_vars)
cont_vars = return_list_of_vars(cont_vars)
logging.info(f"categorical variables: \n {cat_vars}")
logging.info(f"continous variables: \n {cont_vars}")

pipe = preprocessing.generate_sklearn_preprocessing_pipeline(
    feature_dict, impute=True, impute_strategy='mean'
)

adls_path = os.path.join((os.path.join(etl['data_lake_path'], 'experiments', experiment_name)
    if experiment
    else os.path.join(etl_dict['data_lake_path'],
    os.environ.get('CI_COMMIT_SHA', 'LocalRunNBS')))
    , models['preprocessors_adls_path'])

model = model_trainer(X_train, X_valid, y_train, y_valid,
    evals=evals, sub=hyper_sub_size, train=train_size)

y_var=[k.upper() for k, v in features.items() if v['input_definition'] == 'LABEL']

"""
Custom needs for each project type this works for a binary classification
this is not my best work, but trying to put something together 
this is dry I am sure i could make this just a few lines 
"""
result_dict = {}
logging.info('Training Set Evaluation')

eval_list_train = evaluate(model, X_train, y_train, y_var, feature_importance=True)
metric1, metric2, metric3, columns, fi_permutation = eval_list_train
result_dict['training_metrics']={k: v for k, v in zip(columns, [metric1]+[metric2]+[metric3])}
result_dict['fi_train']={k:v for k, v in fi_permutation[:10].values}
logging.info('Validation Set Evaluation')
eval_list_valid = evaluate(model, X_valid, y_valid, y_var, feature_importance=True)
metric1, metric2, metric3, columns, fi_permutation = eval_list_train
result_dict['valid_metrics']={k: v for k, v in zip(columns, [metric1]+[metric2]+[metric3])}
result_dict['fi_valid']={k:v for k, v in fi_permutation[:10].values}
if X_test is not None:
    logging.info('Test Set Evaluation')
    eval_list_test = evaluate(model, X_test, y_test, y_var, feature_importance=True)
    metric1, metric2, metric3, columns, fi_permutation = eval_list_train
    result_dict['test_metrics']={k: v for k, v in zip(columns, [metric1]+[metric2]+[metric3])}
    result_dict['fi_test']={k:v for k, v in fi_permutation[:10].values}
    
sf = snowflake_query(sfSchema='LTBP')
send_holdout_results_to_sf(sf, id_list, y_pred_proba, experiment, experiment_name, etl, models)

adls_path = os.path.join((os.path.join(etl['data_lake_path'], 'experiments', experiment_name)
    if experiment 
    else os.path.join(etl_dict['data_lake_path'], 
    os.environ.get('CI_COMMIT_SHA', 'LocalRunNBS'))))

project_log_df = project_log.project_log(
    snowflake_connection=sf,
    table_name=models['tracking_table'],
    action_description=models[experiment_name]["description"],
    transaction_type="model_training",
    artifacts=json.dumps({"azure_parent_folder": adls_path}),
    metrics=json.dumps(result_dict),
    append_or_replace="append",
)
logging.info(f'project log preview:\n{project_log_df.head(2)}')

INFO:data_system_utilities.snowflake.utils:stage_query: 
 create or replace stage ltbpFY23LocalRunTest
url='azure://vaildtscadls.blob.core.windows.net/vailadls/projects/LTBP/FY23/experiments/BASELINE'
credentials=(azure_sas_token='**MASKED**')
encryption=(type= 'NONE')
file_format = (type = parquet        )
INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utilities.snowflake.query:executing query
INFO:data_system_utilities.snowflake.query:data loaded from snowflake
INFO:data_system_utilities.snowflake.query:connection to snowflake has been turned off
INFO:data_system_utilities.snowflake.query:Stage area LTBPFY23LOCALRUNTEST successfully created.
INFO:root:adls snowflake stage query 
    select
    $1:"ECID"::varchar as ECID
, $1:"SEASONYEAR"::varchar as SEASONYEAR
, $1:"AGE"::varchar as AGE
, $1:"AVGVISITPERSEASON"::varchar as AVGVISITPERSEASON
, $1:"BOUGHTPASS"::varchar as BOUGHTPASS
, $1:"DESTINATIONGEOAFINITYLABEL"::varchar as DESTIN

INFO:machine_learning_utilities.preprocessing:Feature: DESTINATIONGEOAFINITYLABEL --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: EVERCOREPASS --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: EVERPASS --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: GENDERCODE --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: GUESTBEHAVIOR --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: ISEPICMIXACTIVATED --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: MARKET

INFO:root:categorical variables: 
 ['DESTINATIONGEOAFINITYLABEL', 'EVERCOREPASS', 'EVERPASS', 'GENDERCODE', 'GUESTBEHAVIOR', 'ISEPICMIXACTIVATED', 'MARKETINGZONE', 'MOSTCOMMONTICKETCOMP', 'MOSTSUBSEASONVISITED', 'MOSTVISITEDREGION', 'MOSTVISITEDRESORT', 'ONLYSINGLERESORTKEY', 'PARTNERRESORTSCANNERFLAG', 'SKIERABILITYLABEL', 'TOTALSEASONSSCANNED', 'VISITMOSTINPEAK']
INFO:root:continous variables: 
 ['AGE', 'AVGVISITPERSEASON', 'RESORTSVISITED', 'SUBSEASONSPERYEAR', 'TOTALVISITS']
INFO:machine_learning_utilities.preprocessing:Creating Sklearn Preprocessing Pipeline
INFO:machine_learning_utilities.preprocessing:Feature: DESTINATIONGEOAFINITYLABEL --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: EVERCOREPASS --> Transformer: OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
INFO:machine_learning_utilities.preprocessing:Feature: EVERPASS --> Transformer: OrdinalEncoder(handle_unkno

  0%|                       | 0/2 [00:00<?, ?trial/s, best loss=?]

INFO:hyperopt.tpe:build_posterior_wrapper took 0.008663 seconds
INFO:hyperopt.tpe:TPE using 0 trials


 50%|▌| 1/2 [00:00<00:00,  1.42trial/s, best loss: 0.1156746118111

INFO:hyperopt.tpe:build_posterior_wrapper took 0.007475 seconds
INFO:hyperopt.tpe:TPE using 1/1 trials with best loss 0.115675


100%|█| 2/2 [00:06<00:00,  3.15s/trial, best loss: 0.0923767434141

INFO:root:Full training on 10000 rows


INFO:root:Training Set Evaluation
INFO:root:Variable(s) of interest ['BOUGHTPASS'] AUC: 0.919    Accuracy: 0.846    Balanced Accuracy: 0.847
INFO:root:Feature Importance df: 
                           COLS     IMP
0                     EVERPASS  0.1740
1         MOSTCOMMONTICKETCOMP  0.0260
2            SUBSEASONSPERYEAR  0.0186
3                          AGE  0.0172
4                GUESTBEHAVIOR  0.0164
5                  TOTALVISITS  0.0162
6                 EVERCOREPASS  0.0152
7         MOSTSUBSEASONVISITED  0.0116
8            MOSTVISITEDRESORT  0.0086
9   DESTINATIONGEOAFINITYLABEL  0.0080
10           MOSTVISITEDREGION  0.0068
11         ONLYSINGLERESORTKEY  0.0054
12          ISEPICMIXACTIVATED  0.0050
13              RESORTSVISITED  0.0044
14           SKIERABILITYLABEL  0.0042
15               MARKETINGZONE  0.0034
16           AVGVISITPERSEASON  0.0034
17         TOTALSEASONSSCANNED  0.0018
18             VISITMOSTINPEAK  0.0016
19                  GENDERCODE  0.0010
20   

In [ ]:
sf.run_sql_str("DROP TABLE LTBP_MODEL_TRACKING_FY23")

INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utilities.snowflake.query:executing query
INFO:data_system_utilities.snowflake.query:data loaded from snowflake
INFO:data_system_utilities.snowflake.query:connection to snowflake has been turned off
INFO:data_system_utilities.snowflake.query:LTBP_MODEL_TRACKING_FY23 successfully dropped.


,status
0,LTBP_MODEL_TRACKING_FY23 successfully dropped.


In [ ]:
#| export

from data_system_utilities.azure.storage import FileHandling
from data_system_utilities.file_parsers import yaml
from data_system_utilities.snowflake.utils import make_stage_query_generator

from machine_learning_utilities import preprocessing

from LTBP.data.utils import snowflake_query, get_yaml_dicts, generate_data_lake_query

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from rfpimp import *

import sklearn.preprocessing as y_transform
import os
import logging
import pickle

In [ ]:
from machine_learning_utilities.project_log import project_log
from data_system_utilities.azure.storage import FileHandling



In [ ]:
sf.run_sql_str("DROP TABLE LTBP_MODEL_TRACKING_FY23")

INFO:data_system_utilities.snowflake.utils:connection to snowflake established...
INFO:data_system_utilities.snowflake.query:executing query
INFO:data_system_utilities.snowflake.query:data loaded from snowflake
INFO:data_system_utilities.snowflake.query:connection to snowflake has been turned off
INFO:data_system_utilities.snowflake.query:LTBP_MODEL_TRACKING_FY23 successfully dropped.


,status
0,LTBP_MODEL_TRACKING_FY23 successfully dropped.


In [ ]:
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)
auc = metrics.roc_auc_score(y_test, y_pred_proba[:, 1])
acc = metrics.accuracy_score(y_test, y_pred)
bacc = metrics.balanced_accuracy_score(y_test, y_pred)
logging.info(f'For Test Set Unseen Data {y_var} AUC: {auc:.3f} Accuracy: {acc:.3f} Balanced Accuracy: {bacc:.3f}')
logging.info(f'Size of Test Data set: {len(y_test)}')

INFO:root:For Test Set Unseen Data ['BOUGHTPASS'] AUC: 0.899 Accuracy: 0.828 Balanced Accuracy: 0.829
INFO:root:Size of Test Data set: 900


INFO:data_system_utilities.snowflake.query:creating table LTBP_MODEL_TRACKING_FY23
INFO:data_system_utilities.snowflake.query:sqlalchemy snowflake engine created
INFO:data_system_utilities.snowflake.query:table created


,ACTION_DESCRIPTION,TRANSACTION_TYPE,COMMITID,ENVIRONMENT,BRANCH,TIMESTAMP,ARTIFACTS,METRICS
0,Standard baseline xgb_hyperopt approach status...,model_training,LocalRunTest,dev,random,2022-10-26 20:27:03.842182-06:00,"{""azure_parent_folder"": ""projects/LTBP/FY23/ex...","{""training_metrics"": {""auc"": 0.902594497729876..."
